In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
import pandas as pd
import joblib
import numpy as np
import json

# Initialize Flask application
app = Flask(__name__)
CORS(app, resources={r"/predict": {"origins": "http://localhost:3000"}})
# Load your trained model
model = joblib.load('ransomware_detection_model.pkl')

# Define the list of features your model expects
selected_features = ['10', '17', '67', '77', '82', '84', '122', '151', '164', '170', '171',
                     '196', '200', '332', '5574', '7121', '11686', '12043', '14267', '14747',
                     '20049', '21986', '24730', '29769']

# Define a route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    # Check if a file was posted
    if 'file' not in request.files:
        return jsonify({'error': 'No file part in the request'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected for uploading'}), 400

    # Read the file into a pandas DataFrame
    data_dict = json.load(file)
    data = pd.DataFrame([data_dict])

    # Check if the data is a DataFrame. If not, convert it into a DataFrame
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame([data])

    # Extract only the selected features from the input data
    filtered_data = {feature: data.get(feature, [0]) for feature in selected_features}  # Use [0] for missing features
    input_df = pd.DataFrame(filtered_data)

    # Make prediction
    prediction = model.predict(input_df)
    probabilities = model.predict_proba(input_df)

    # Get the probability of the predicted class
    predicted_class_index = np.argmax(probabilities, axis=1)[0]
    predicted_probability = probabilities[0][predicted_class_index]

    # Return the prediction as a JSON response
    return jsonify({
        'prediction': prediction.tolist(),
        'probability': predicted_probability
    })

# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


c:\Users\mrela\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mrela\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2024 23:59:07] "OPTIONS /predict HTTP/1.1" 200 -
127.